In [4]:
pip install torch

In [5]:
import torch

In [6]:
word=open('names.txt','r').read().splitlines()

In [7]:
b={}
for w in word:
    chs=['<S>']+list(w)+['<E>']
    for ch1,ch2 in zip(chs,chs[1:]):
        bigram=(ch1,ch2)
        b[bigram]=b.get(bigram,0)+1


In [8]:
sorted(b.items(),key=lambda kv : -kv[1])

[(('n', '<E>'), 6763),
 (('a', '<E>'), 6640),
 (('a', 'n'), 5438),
 (('<S>', 'a'), 4410),
 (('e', '<E>'), 3983),
 (('a', 'r'), 3264),
 (('e', 'l'), 3248),
 (('r', 'i'), 3033),
 (('n', 'a'), 2977),
 (('<S>', 'k'), 2963),
 (('l', 'e'), 2921),
 (('e', 'n'), 2675),
 (('l', 'a'), 2623),
 (('m', 'a'), 2590),
 (('<S>', 'm'), 2538),
 (('a', 'l'), 2528),
 (('i', '<E>'), 2489),
 (('l', 'i'), 2480),
 (('i', 'a'), 2445),
 (('<S>', 'j'), 2422),
 (('o', 'n'), 2411),
 (('h', '<E>'), 2409),
 (('r', 'a'), 2356),
 (('a', 'h'), 2332),
 (('h', 'a'), 2244),
 (('y', 'a'), 2143),
 (('i', 'n'), 2126),
 (('<S>', 's'), 2055),
 (('a', 'y'), 2050),
 (('y', '<E>'), 2007),
 (('e', 'r'), 1958),
 (('n', 'n'), 1906),
 (('y', 'n'), 1826),
 (('k', 'a'), 1731),
 (('n', 'i'), 1725),
 (('r', 'e'), 1697),
 (('<S>', 'd'), 1690),
 (('i', 'e'), 1653),
 (('a', 'i'), 1650),
 (('<S>', 'r'), 1639),
 (('a', 'm'), 1634),
 (('l', 'y'), 1588),
 (('<S>', 'l'), 1572),
 (('<S>', 'c'), 1542),
 (('<S>', 'e'), 1531),
 (('j', 'a'), 1473),
 (

In [9]:
N=torch.zeros((27,27),dtype=torch.int32)

In [10]:
chars=sorted(list(set(''.join(word))))

In [11]:
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos={i:s for s,i in  stoi.items()}

In [12]:
b={}
for w in word:
    chs=['.']+list(w)+['.']
    for ch1,ch2 in zip(chs,chs[1:]):
      ix1=stoi[ch1]
      ix2=stoi[ch2]
      N[ix1,ix2]+=1


In [13]:
N

tensor([[   0, 4410, 1306, 1542, 1690, 1531,  417,  669,  874,  591, 2422, 2963,
         1572, 2538, 1146,  394,  515,   92, 1639, 2055, 1308,   78,  376,  307,
          134,  535,  929],
        [6640,  556,  541,  470, 1042,  692,  134,  168, 2332, 1650,  175,  568,
         2528, 1634, 5438,   63,   82,   60, 3264, 1118,  687,  381,  834,  161,
          182, 2050,  435],
        [ 114,  321,   38,    1,   65,  655,    0,    0,   41,  217,    1,    0,
          103,    0,    4,  105,    0,    0,  842,    8,    2,   45,    0,    0,
            0,   83,    0],
        [  97,  815,    0,   42,    1,  551,    0,    2,  664,  271,    3,  316,
          116,    0,    0,  380,    1,   11,   76,    5,   35,   35,    0,    0,
            3,  104,    4],
        [ 516, 1303,    1,    3,  149, 1283,    5,   25,  118,  674,    9,    3,
           60,   30,   31,  378,    0,    1,  424,   29,    4,   92,   17,   23,
            0,  317,    1],
        [3983,  679,  121,  153,  384, 1271,   82,

In [14]:
p=N[0].float()
p=p/p.sum()
p

tensor([0.0000, 0.1377, 0.0408, 0.0481, 0.0528, 0.0478, 0.0130, 0.0209, 0.0273,
        0.0184, 0.0756, 0.0925, 0.0491, 0.0792, 0.0358, 0.0123, 0.0161, 0.0029,
        0.0512, 0.0642, 0.0408, 0.0024, 0.0117, 0.0096, 0.0042, 0.0167, 0.0290])

In [15]:
g=torch.Generator().manual_seed(2147483647)
ix=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
ix

10

In [16]:
P=(N+1).float()
P/=P.sum(1,keepdim=True)

In [17]:
#WONT WORK
#  27 27
#  1  27   (as we are not keeping dims = True)

# P=N.float()
# P=P/P.sum(1)
# P[:,0].sum()

In [18]:
g=torch.Generator().manual_seed(2147483697)
for _ in range(2):
  ix=0
  out = []
  while True:
    p=P[ix]
    ix=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
    out.append(itos[ix])
    if ix == 0 :
      break
  print(''.join(out))

braara.
ke.




```
 GOAL : maximize the likelihood of data w.r.t model parameters(statistical modelling)
 equivalent to maximizing the log likelihood because log is monotonic
 equivalent to minimizing the negative log likelihood
 equivalent to minimizing the average negative log likelihood
```



In [19]:
#likelihood is the product of all of these probabilities
# log(a*b*c)= log(a)+log(b)+log(c)

b={}
log_likelihood=0.0
n=0
for w in word:
    chs=['.']+list(w)+['.']
    for ch1,ch2 in zip(chs,chs[1:]):
      ix1=stoi[ch1]
      ix2=stoi[ch2]
      prob=P[ix1,ix2]
      logprob=torch.log(prob)
      log_likelihood+=logprob
      n+=1
      # print(f"{ch1}{ch2} , {prob:.4f} , {logprob:.4f}")
print(log_likelihood)
nil=-log_likelihood
print(nil)
print(nil/n)

tensor(-559951.5625)
tensor(559951.5625)
tensor(2.4544)


###TRIGRAM

In [20]:
NN=torch.zeros((27,27,27),dtype=torch.int32)

In [21]:
b={}
for w in word:
    chs=['.']+list(w)+['.']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
      ix1=stoi[ch1]
      ix2=stoi[ch2]
      ix3=stoi[ch3]
      NN[ix1,ix2,ix3]+=1


In [22]:
NN

tensor([[[  0,   0,   0,  ...,   0,   0,   0],
         [  0, 207, 190,  ...,  27, 173, 152],
         [  0, 169,   0,  ...,   0,   4,   0],
         ...,
         [  0,  57,   0,  ...,   1,  17,  11],
         [  0, 246,   0,  ...,   0,   0,   2],
         [  0, 456,   0,  ...,   0,  91,   1]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [ 40,   0,   5,  ...,   0,  20,  11],
         [ 36,  28,  20,  ...,   0,  12,   0],
         ...,
         [ 11,   5,   0,  ...,  17,   6,   3],
         [163, 389,  13,  ...,   0,  16,  40],
         [ 38, 123,   0,  ...,   0,  12,  22]],

        [[  0,   0,   0,  ...,   0,   0,   0],
         [ 46,   5,   5,  ...,   4,  31,   4],
         [  1,   8,   0,  ...,   0,   9,   0],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [ 55,   4,   1,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        ...,

        [[  0,   0,   0,  ...,   0,   0,   0],
         [ 10,   0,   2,  ...,   0,  10,   0]

In [23]:
P=(NN+1).float()
P/=P.sum(2,keepdim=True)

In [24]:
g=torch.Generator().manual_seed(2147483697)
for _ in range(10):
  ix1=0
  ix2=1
  out = []
  while True:
    p=P[ix1][ix2]
    ix3=torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
    out.append(itos[ix3])
    if ix3 == 0 :
      break
    ix1=ix2
    ix2=ix3
  print(''.join(out))

braira.
de.
h.
jiukie.
h.
n.
mot.
ry.
ydissisamarlethiksvy.
yafitaliona.


In [25]:
#likelihood is the product of all of these probabilities
# log(a*b*c)= log(a)+log(b)+log(c)

b={}
log_likelihood=0.0
n=0
for w in word:
    chs=['.']+list(w)+['.']
    for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
      ix1=stoi[ch1]
      ix2=stoi[ch2]
      ix3=stoi[ch3]
      prob=P[ix1,ix2,ix3]
      logprob=torch.log(prob)
      log_likelihood+=logprob
      n+=1
      # print(f"{ch1}{ch2} , {prob:.4f} , {logprob:.4f}")
print(log_likelihood)
nil=-log_likelihood
print(nil)
print(nil/n)

tensor(-410414.9688)
tensor(410414.9688)
tensor(2.0927)


NEURAL NETWORK

In [26]:
#create the trainig set of all bigrams(x,y)
xs,ys=[] ,[]
for w in word[:1]:
    chs=['.']+list(w)+['.']
    for ch1,ch2 in zip(chs,chs[1:]):
      # print(ch1,ch2)
      ix1=stoi[ch1]
      ix2=stoi[ch2]
      xs.append(ix1)
      ys.append(ix2)

xs=torch.tensor(xs)
ys=torch.tensor(ys)

# print(xs,ys)




In [27]:
import torch.nn.functional as F
xenc=F.one_hot(xs,num_classes=27).float()  # we want to feed  floating values to the neural network

In [28]:
xenc.shape

torch.Size([5, 27])

In [29]:
W= torch.randn((27,27))
xenc@W

tensor([[ 1.0950,  0.6521,  0.7953,  0.8294,  0.3932,  1.2772,  1.0585, -0.8130,
         -0.0170,  1.5280,  1.0834,  0.0530,  0.9910,  0.8806, -0.3309,  0.6359,
          1.8753, -0.9402,  0.6251,  0.0173, -1.0657,  1.0862,  0.1646,  0.0140,
         -1.2127,  0.1658,  0.8886],
        [-0.8528, -1.3825, -0.5517,  1.3323, -0.4966, -0.0708, -0.3837,  0.9186,
          0.6016,  0.8046,  1.8429,  0.9324,  1.8090, -0.4814,  2.9916, -0.5573,
          0.2175, -0.9438,  0.5021,  0.8810,  0.3784, -2.5708, -2.5306, -0.4170,
         -0.2810, -0.8524, -1.6967],
        [-2.4856, -0.0637, -1.6918,  0.7430,  0.1133, -0.7453, -0.1503, -1.6353,
         -0.1197, -1.0830,  1.1927, -1.7150, -0.8994,  1.0229, -0.0757, -0.3056,
          0.6616,  0.9152, -1.5670, -0.7620,  0.1513,  1.3391, -0.8892, -1.3594,
          0.0759,  0.4193,  1.4833],
        [-2.4856, -0.0637, -1.6918,  0.7430,  0.1133, -0.7453, -0.1503, -1.6353,
         -0.1197, -1.0830,  1.1927, -1.7150, -0.8994,  1.0229, -0.0757, -0.3056

In [30]:
logits=xenc@W  # log counts

counts=logits.exp()
prob=counts/counts.sum(1,keepdim=True)

In [31]:
prob[0].sum()

tensor(1.)

In [32]:
xs

tensor([ 0,  5, 13, 13,  1])

In [33]:
ys

tensor([ 5, 13, 13,  1,  0])

In [34]:
# randomly initialize 27 neurons weight ,each neuron receives 27  inputs
g=torch.Generator().manual_seed(2147483647)
W=torch.randn((27,27),generator=g)

In [35]:
xenc=F.one_hot(xs,num_classes=27).float() # input to the network : one-hot encoding
logits=xenc@W  # predict log counts
counts=logits.exp() # counts
probs=counts/counts.sum(1,keepdim=True)



In [36]:
probs.shape

torch.Size([5, 27])

In [37]:
nlls=torch.zeros(5)
for  i in range(5):
  x=xs[i].item() # input character index
  y=ys[i].item() # label character index

  print('-----------')
  print(f'bigram example {i+1}: {itos[x]} {itos[y]}, indexes: {x}{y}')
  print('input to the neural net :', x)
  print('output probabilities from the neural net :', probs[i])
  print('label(actual next character):',y)
  p=probs[i,y]
  print('probability assigned by the net to the correct character :',p.item())
  logp=torch.log(p)
  print('log likelihood :',logp.item())
  nll=-logp
  print('negative log likelihood',nll.item())
  nlls[i]=nll


  print("=======================================================")
  print('average negative log likelihood is :',nlls.mean().item())



-----------
bigram example 1: . e, indexes: 05
input to the neural net : 0
output probabilities from the neural net : tensor([0.0607, 0.0100, 0.0123, 0.0042, 0.0168, 0.0123, 0.0027, 0.0232, 0.0137,
        0.0313, 0.0079, 0.0278, 0.0091, 0.0082, 0.0500, 0.2378, 0.0603, 0.0025,
        0.0249, 0.0055, 0.0339, 0.0109, 0.0029, 0.0198, 0.0118, 0.1537, 0.1459])
label(actual next character): 5
probability assigned by the net to the correct character : 0.01228625513613224
log likelihood : -4.399273872375488
negative log likelihood 4.399273872375488
average negative log likelihood is : 0.8798547983169556
-----------
bigram example 2: e m, indexes: 513
input to the neural net : 5
output probabilities from the neural net : tensor([0.0290, 0.0796, 0.0248, 0.0521, 0.1989, 0.0289, 0.0094, 0.0335, 0.0097,
        0.0301, 0.0702, 0.0228, 0.0115, 0.0181, 0.0108, 0.0315, 0.0291, 0.0045,
        0.0916, 0.0215, 0.0486, 0.0300, 0.0501, 0.0027, 0.0118, 0.0022, 0.0472])
label(actual next character): 13
pro

In [38]:
# randomly initialize 27 neurons weight ,each neuron receives 27  inputs
g=torch.Generator().manual_seed(2147483647)
W=torch.randn((27,27),generator=g,requires_grad=True)

In [39]:
#forward pass
xenc=F.one_hot(xs,num_classes=27).float() # input to the network : one-hot encoding
logits=xenc@W  # predict log counts
counts=logits.exp() # counts
probs=counts/counts.sum(1,keepdim=True)
loss=-probs[torch.arange(5),ys].log().mean()


In [40]:
loss

tensor(3.7693, grad_fn=<NegBackward0>)

In [41]:
#backward pass
W.grad=None #set to zero
loss.backward()

In [42]:
print(W.grad.shape)

torch.Size([27, 27])


In [43]:
# update

W.data+=-0.1*W.grad

In [44]:
loss

tensor(3.7693, grad_fn=<NegBackward0>)

In [45]:
#create the trainig set of all bigrams(x,y)
xs,ys=[] ,[]
for w in word:
    chs=['.']+list(w)+['.']
    for ch1,ch2 in zip(chs,chs[1:]):
      # print(ch1,ch2)
      ix1=stoi[ch1]
      ix2=stoi[ch2]
      xs.append(ix1)
      ys.append(ix2)

xs=torch.tensor(xs)
ys=torch.tensor(ys)
num=xs.nelement()
print('number of examples:',num)

# print(xs,ys)

number of examples: 228146


In [50]:
# randomly initialize 27 neurons weight ,each neuron receives 27  inputs
g=torch.Generator().manual_seed(2147483647)
W=torch.randn((27,27),generator=g,requires_grad=True)

xenc=F.one_hot(xs,num_classes=27).float() # input to the network : one-hot encoding



In [47]:
for i in range(1000):
  # forward pass
  logits=xenc@W  # predict log counts
  counts=logits.exp() # counts
  probs=counts/counts.sum(1,keepdim=True)
  loss=-probs[torch.arange(num),ys].log().mean() + 0.01 * (W**2).mean()

  #backward pass
  W.grad=None #set to zero
  loss.backward()

  # update

  W.data+=-2*W.grad
  if i%10 == 0:
    print(f"Loss at the {i}th iteration is: {loss}\n")

Loss at the 0th iteration is: 3.7686190605163574

Loss at the 10th iteration is: 3.6029648780822754

Loss at the 20th iteration is: 3.4683403968811035

Loss at the 30th iteration is: 3.357903242111206

Loss at the 40th iteration is: 3.2670750617980957

Loss at the 50th iteration is: 3.191946268081665

Loss at the 60th iteration is: 3.1290879249572754

Loss at the 70th iteration is: 3.075745105743408

Loss at the 80th iteration is: 3.029876470565796

Loss at the 90th iteration is: 2.9900121688842773

Loss at the 100th iteration is: 2.955076217651367

Loss at the 110th iteration is: 2.9242498874664307

Loss at the 120th iteration is: 2.8968865871429443

Loss at the 130th iteration is: 2.872462272644043

Loss at the 140th iteration is: 2.8505427837371826

Loss at the 150th iteration is: 2.83076810836792

Loss at the 160th iteration is: 2.812838554382324

Loss at the 170th iteration is: 2.7965023517608643

Loss at the 180th iteration is: 2.7815499305725098

Loss at the 190th iteration is: 

In [48]:
g=torch.Generator().manual_seed(214483647)
for i in range(5):
  out=[]
  ix=0

  while True:
    # forward pass
    xenc=F.one_hot(torch.tensor([ix]),num_classes=27).float()
    logits=xenc@W  # predict log counts
    counts=logits.exp() # counts
    probs=counts/counts.sum(1,keepdim=True)

    ix=torch.multinomial(probs,num_samples=1,replacement=True,generator=g).item()
    out.append(itos[ix])

    if ix==0:
      break


  print(''.join(out))


kelen.
nxzanan.
o.
ayar.
riman.


In [ ]:
for i in range(1000):
  # forward pass
  logits=xenc@W  # predict log counts
  counts=logits.exp() # counts
  probs=counts/counts.sum(1,keepdim=True)

  # Computing counts = logits.exp() and then normalizing can overflow if any logit is large
  #so we can use pytorch's inbuilt cross-entropy function
  loss=F.cross_entropy(logits,ys) + 0.01 * (W**2).mean()

  #backward pass
  W.grad=None #set to zero
  loss.backward()

  # update

  W.data+=-2*W.grad
  if i%10 == 0:
    print(f"Loss at the {i}th iteration is: {loss}\n")

# Instead of using one-hot encoding , we directly index into the row!

In [51]:
# randomly initialize 27 neurons weight ,each neuron receives 27  inputs
g=torch.Generator().manual_seed(2147483647)
W=torch.randn((27,27),generator=g,requires_grad=True)

In [55]:
for i in range(1000):

""" When you do

logits = torch.tensor(W[xs], dtype=torch.float32)

it forces PyTorch to copy the data in W[xs] into a brand‐new tensor that has no connection to the original W in the autograd graph. In other words:

    W[xs] by itself is a “view” of W (or at least a tensor that still tracks gradients back to W).

    But wrapping it in torch.tensor(…) takes its raw data (a NumPy‐style array underneath) and creates a fresh tensor with requires_grad=False. That newly created tensor has no “history,” so when you do loss.backward(), nothing flows back into W.

As a result, even though you later call loss.backward(), W.grad will remain None (or zero), because your logits was built from a detached copy. Thus the model never learns.
"""
  logits=W[xs]  # predict log counts

  loss=F.cross_entropy(logits,ys) + 0.01 * (W**2).mean()

  #backward pass
  W.grad=None #set to zero
  loss.backward()

  # update

  W.data+=-2*W.grad
  if i%10 == 0:
    print(f"Loss at the {i}th iteration is: {loss}\n")

Loss at the 0th iteration is: 3.7201380729675293

Loss at the 10th iteration is: 3.561170816421509

Loss at the 20th iteration is: 3.4315710067749023

Loss at the 30th iteration is: 3.32521653175354

Loss at the 40th iteration is: 3.2377700805664062

Loss at the 50th iteration is: 3.165433168411255

Loss at the 60th iteration is: 3.1048848628997803

Loss at the 70th iteration is: 3.0534780025482178

Loss at the 80th iteration is: 3.009258270263672

Loss at the 90th iteration is: 2.9708192348480225

Loss at the 100th iteration is: 2.937129020690918

Loss at the 110th iteration is: 2.907400131225586

Loss at the 120th iteration is: 2.8810102939605713

Loss at the 130th iteration is: 2.8574540615081787

Loss at the 140th iteration is: 2.836313247680664

Loss at the 150th iteration is: 2.8172404766082764

Loss at the 160th iteration is: 2.799947738647461

Loss at the 170th iteration is: 2.7841923236846924

Loss at the 180th iteration is: 2.769773006439209

Loss at the 190th iteration is: 2